<h1> Stack Overflow Survey Analysis</h1>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import csv
from collections import defaultdict, Counter

In [2]:
survey = pd.read_csv("data/survey_results_public.csv")

In [4]:
survey.head()

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
0,1,I am a student who is learning to code,Yes,Never,The quality of OSS and closed source software ...,"Not employed, and not looking for work",United Kingdom,No,Primary/elementary school,NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,14.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
1,2,I am a student who is learning to code,No,Less than once per year,The quality of OSS and closed source software ...,"Not employed, but looking for work",Bosnia and Herzegovina,"Yes, full-time","Secondary school (e.g. American high school, G...",NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,19.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
2,3,"I am not primarily a developer, but I write co...",Yes,Never,The quality of OSS and closed source software ...,Employed full-time,Thailand,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Web development or web design,...,Just as welcome now as I felt last year,Tech meetups or events in your area;Courses on...,28.0,Man,No,Straight / Heterosexual,NaN,Yes,Appropriate in length,Neither easy nor difficult
3,4,I am a developer by profession,No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,22.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Easy
4,5,I am a developer by profession,Yes,Once a month or more often,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,Ukraine,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech meetups or events in your area;Courses on...,30.0,Man,No,Straight / Heterosexual,White or of European descent;Multiracial,No,Appropriate in length,Easy


In [5]:
survey.shape

(88883, 85)

In [6]:
survey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88883 entries, 0 to 88882
Data columns (total 85 columns):
Respondent                88883 non-null int64
MainBranch                88331 non-null object
Hobbyist                  88883 non-null object
OpenSourcer               88883 non-null object
OpenSource                86842 non-null object
Employment                87181 non-null object
Country                   88751 non-null object
Student                   87014 non-null object
EdLevel                   86390 non-null object
UndergradMajor            75614 non-null object
EduOther                  84260 non-null object
OrgSize                   71791 non-null object
DevType                   81335 non-null object
YearsCode                 87938 non-null object
Age1stCode                87634 non-null object
YearsCodePro              74331 non-null object
CareerSat                 72847 non-null object
JobSat                    70988 non-null object
MgrIdiot                  61

In [7]:
survey['Hobbyist'].value_counts()/len(survey['Hobbyist'])

Yes    0.801694
No     0.198306
Name: Hobbyist, dtype: float64

In [ ]:
with open("data/survey_results_public.csv", 'r', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)
    counts = Counter()
    for line in csv_reader:
        counts[line['Hobbyist']]+=1
total = counts['Yes']+counts['No']
yes_cnt = round((counts['Yes']/total)*100,2)
no_cnt = round((counts['No']/total)*100,2)
print(yes_cnt)
print(no_cnt)

80.17
19.83


In [32]:
language_counter = Counter()
languages = survey['LanguageWorkedWith']
nulls = languages.isnull().sum()
total = languages.shape[0]-nulls
for answer in languages:
    try:
        split_ans = answer.split(';')
    except:
        split_ans = [str(answer)]
    language_counter.update(split_ans)
for lan, val in language_counter.most_common(5):
    print(f"{lan}: {round(val/total * 100 , 2)}%")

JavaScript: 67.63%
HTML/CSS: 63.34%
SQL: 54.29%
Python: 41.62%
Java: 41.02%


In [33]:
import pandas_profiling

In [34]:
pandas_profiling.ProfileReport(survey)

C:\Users\sahana\Anaconda3\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,85
Number of observations,88883
Total Missing (%),13.6%
Total size in memory,57.6 MiB
Average record size in memory,680.0 B
Numeric,6
Categorical,79
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [35]:
lan_dev_type = survey[['DevType', 'LanguageWorkedWith']]

lan_dev_type = lan_dev_type.dropna(subset=['DevType']).reindex()

lan_dev_type['LanguageWorkedWith'] = lan_dev_type['LanguageWorkedWith'].fillna(
    'NA')

dev_type_info = {}


for dev_types, languages in zip(lan_dev_type['DevType'], lan_dev_type['LanguageWorkedWith']):
    dev_type = dev_types.split(';')
    language = languages.split(';')
    for each_dev_type in dev_type:
        dev_type_info.setdefault(each_dev_type, {
            'total': 0,
            'language_counter': Counter()
        })
        dev_type_info[each_dev_type]['language_counter'].update(language)
        dev_type_info[each_dev_type]['total'] += 1


for dev_type, info in dev_type_info.items():
    print(dev_type)
    for language, value in info['language_counter'].most_common(5):
        language_pct = (value / info['total']) * 100
        language_pct = round(language_pct, 2)

        print(f"\t{language}: {language_pct}%")
    print()

Developer, desktop or enterprise applications
	JavaScript: 67.84%
	HTML/CSS: 64.55%
	SQL: 63.56%
	C#: 53.69%
	Java: 44.69%

Developer, front-end
	JavaScript: 87.72%
	HTML/CSS: 83.62%
	SQL: 58.65%
	Java: 37.6%
	PHP: 35.94%

Designer
	HTML/CSS: 78.88%
	JavaScript: 78.33%
	SQL: 60.18%
	PHP: 40.23%
	Java: 39.44%

Developer, back-end
	JavaScript: 72.23%
	HTML/CSS: 65.42%
	SQL: 64.01%
	Java: 44.03%
	Python: 40.67%

Developer, full-stack
	JavaScript: 86.15%
	HTML/CSS: 78.94%
	SQL: 65.54%
	Java: 40.74%
	Bash/Shell/PowerShell: 37.91%

Academic researcher
	Python: 61.06%
	HTML/CSS: 55.87%
	JavaScript: 54.25%
	SQL: 47.55%
	Java: 42.26%

Developer, mobile
	JavaScript: 67.72%
	HTML/CSS: 62.46%
	Java: 57.21%
	SQL: 51.27%
	C#: 34.34%

Data or business analyst
	SQL: 73.88%
	HTML/CSS: 62.11%
	JavaScript: 61.33%
	Python: 51.86%
	Bash/Shell/PowerShell: 38.43%

Data scientist or machine learning specialist
	Python: 79.33%
	SQL: 58.44%
	JavaScript: 51.38%
	HTML/CSS: 50.43%
	Bash/Shell/PowerShell: 44.49%

D